# Coastal change in African countries

* **Products used:** [DE Africa Coastlines]()



## Background

Coastal changes 

## Description

Coastlines data is downloaded and used for analysis so its not impacted by limited number of features returned by WFS.

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

### Load packages
Import Python packages that are used for the analysis.

In [1]:
import geopandas as gpd
import pandas as pd
import rioxarray as rxr
import numpy as np
from deafrica_tools.spatial import xr_rasterize, xr_vectorize
from deafrica_tools.coastal import get_coastlines
#from deafrica_tools.plotting import display_map
from datacube.utils.geometry import Geometry

from matplotlib import pyplot as plt
import matplotlib as mpl


/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [2]:
coastlines = gpd.read_file("~/dev/data/deafricacoastlines_v0.4.0_gadm_level1.gpkg")

In [ ]:
coastlines = coastlines[(coastlines.certainty=='good') & (coastlines.sig_time<0.01) & (coastlines.rate_time<-1)]
admins = coastlines['GID_1'].unique()

In [22]:
admins = admins[[a.startswith("GHA") for a in admins]]

In [23]:
len(admins)

4

In [24]:
import os
import datacube

os.environ['AWS_DEFAULT_REGION']="us-east-1"
os.environ['AWS_S3_ENDPOINT']="s3.us-east-1.amazonaws.com"

# get hotspots and population
# Load population data
ds = rxr.open_rasterio('s3://dataforgood-fb-data/hrsl-cogs/hrsl_general/hrsl_general-latest.vrt', 
                       chunks ={'x': 1000, 'y': 1000})
ds = ds.rename({'x':'longitude','y':'latitude'})


dc=datacube.Datacube()
mrvbf = dc.load(product="dem_srtm_deriv", measurements=["mrvbf","slope"], dask_chunks={'longitude':2000, 'latitude':2000}, 
                like=ds.geobox).mrvbf.squeeze()

In [80]:
for admin in admins:
    if admin == '':continue
    print(admin)
    
    buffered = coastlines[coastlines['GID_1']==admin].copy()
    buffer=500
    buffered['geometry'] = buffered.to_crs('6933').buffer(buffer).to_crs('4326')
    
    coast_bbox = buffered.unary_union.bounds
    
    os.environ['AWS_DEFAULT_REGION']="us-east-1"
    os.environ['AWS_S3_ENDPOINT']="s3.us-east-1.amazonaws.com"
    pop_raster = ds.isel(band=0).sel(longitude=slice(coast_bbox[0],coast_bbox[2]), latitude = slice(coast_bbox[3], coast_bbox[1])).compute()#.astype(int)
    print(pop_raster.shape)
    
    
    os.environ['AWS_DEFAULT_REGION']="af-south-1"
    os.environ['AWS_S3_ENDPOINT']="s3.af-south-1.amazonaws.com"
    mrvbf_raster = mrvbf.sel(longitude=slice(coast_bbox[0],coast_bbox[2]), latitude = slice(coast_bbox[3], coast_bbox[1])).compute()

    hotspots_mask = xr_rasterize(gdf=buffered,
                                da=pop_raster,
                                  transform=pop_raster.geobox.transform,
                                  crs=pop_raster.geobox.crs)
    hotspots_map = pop_raster*(mrvbf_raster==7)*hotspots_mask
    
    nhotspots = (hotspots_map>0).sum()
    if nhotspots>0:
        print("# of hotspot pixels", nhotspots)
        risk_vectors = xr_vectorize(hotspots_map, mask=hotspots_map>0, attribute_col='pop', crs='epsg:4326', export_shp=f'coastchange_hotspots/{admin}_hotspots.geojson')
        break
    else:
        print("not hotspots")
        
    del pop_raster, mrvbf_raster, hotspots_mask, hotspots_map

GHA15_2
(634, 2906)
not hotspots
GHA5_2
(1455, 3961)
not hotspots
GHA7_2
(1132, 3890)
# of hotspot pixels <xarray.DataArray ()>
array(1057)
Coordinates:
    band         int64 1
    time         datetime64[ns] 2000-01-01T12:00:00
    spatial_ref  int32 4326


In [91]:
(np.array([0.99])).astype(int)

array([0])

In [87]:
(hotspots_map>1).sum()

<xarray.DataArray ()>
array(827)
Coordinates:
    band         int64 1
    time         datetime64[ns] 2000-01-01T12:00:00
    spatial_ref  int32 4326

In [103]:
import glob

hotspots_files = glob.glob('coastchange_hotspots/*_hotspots.geojson')

In [104]:
hotspots=[]

for hotspots_file in hotspots_files:
        hotspots.append(gpd.read_file(hotspots_file))

In [105]:
hotspots_all = pd.concat(hotspots)

In [106]:
hotspots_all= hotspots_all.to_crs("EPSG:6933")

In [107]:
hotspots_all['total_pop']=hotspots_all.area*hotspots_all["pop"]/30/30

In [108]:
hotspots_all.sum()

pop           24766.267122
total_pop    227632.525617
dtype: float64

In [109]:
hotspots_pop1000 = hotspots_all#[hotspots_all['total_pop']>1000]

In [110]:
gadm_africa  = gpd.read_file("~/dev/data/gadm_africa.gpkg")

In [111]:
gadm_level0 = gadm_africa.dissolve(by=f"GID_0", as_index=False)
gadm_level0 = gadm_level0.to_crs(hotspots_pop1000.crs)
gadm_level1 = gadm_africa.dissolve(by=f"GID_1", as_index=False)
gadm_level1 = gadm_level1.to_crs(hotspots_pop1000.crs)

In [112]:
gadm_level1#[["NAME_0","geometry"]]

,GID_1,geometry,UID,GID_0,NAME_0,VARNAME_0,NAME_1,VARNAME_1,NL_NAME_1,ISO_1,...,GOVERNEDBY,SOVEREIGN,DISPUTEDBY,REGION,VARREGION,COUNTRY,CONTINENT,SUBCONT,layer,path
0,,"MULTIPOLYGON (((-515142.952 4319024.815, -5151...",103975,GIB,Gibraltar,,,,,,...,,United Kingdom,Spain,,,Gibraltar,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...
1,AGO.10_1,"POLYGON ((1337578.897 -2038881.079, 1335697.89...",2574,AGO,Angola,,Huíla,,,,...,,Angola,,,,Angola,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...
2,AGO.11_1,"MULTIPOLYGON (((1280067.856 -1166669.344, 1278...",2609,AGO,Angola,,Luanda,Loanda,,AO-LUA,...,,Angola,,,,Angola,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...
3,AGO.12_1,"POLYGON ((1874305.731 -1271982.925, 1871843.73...",2636,AGO,Angola,,Lunda Norte,,,,...,,Angola,,,,Angola,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...
4,AGO.13_1,"POLYGON ((1928191.690 -1428331.723, 1927307.78...",2662,AGO,Angola,,Lunda Sul,,,,...,,Angola,,,,Angola,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,ZWE.5_1,"POLYGON ((3020295.985 -2404184.844, 3020302.64...",355148,ZWE,Zimbabwe,,Mashonaland East,,,ZW-ME,...,,Zimbabwe,,,,Zimbabwe,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...
879,ZWE.6_1,"POLYGON ((2853604.495 -2303221.475, 2853221.16...",355377,ZWE,Zimbabwe,,Mashonaland West,,,ZW-MW,...,,Zimbabwe,,,,Zimbabwe,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...
880,ZWE.7_1,"POLYGON ((2944204.594 -2705624.596, 2944193.69...",355607,ZWE,Zimbabwe,,Masvingo,,,ZW-MV,...,,Zimbabwe,,,,Zimbabwe,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...
881,ZWE.8_1,"POLYGON ((2508219.371 -2384702.939, 2508225.83...",355850,ZWE,Zimbabwe,,Matabeleland North,,,ZW-MN,...,,Zimbabwe,,,,Zimbabwe,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...


In [113]:
hotspots_pop1000=hotspots_pop1000.sjoin_nearest(gadm_level0[["GID_0", "NAME_0","geometry"]], how="left", max_distance=1000)

In [114]:
hotspots_pop1000 = hotspots_pop1000.drop(columns=["index_right"])

In [115]:
hotspots_pop1000=hotspots_pop1000.sjoin_nearest(gadm_level1[["GID_1", "NAME_1","geometry"]], how="left", max_distance=1000)

In [116]:
hotspots_pop1000 = hotspots_pop1000.drop(columns=["index_right"])
hotspots_pop1000.to_crs("EPSG:4326").to_file('coastchange_hotspots/hotspots_pop_1000.geojson')

In [117]:
hotspots_pop1000_buffered =hotspots_pop1000.copy()
hotspots_pop1000_buffered['geometry'] = hotspots_pop1000_buffered.buffer(50000)

In [118]:
hotspots_pop1000_buffered.to_crs("EPSG:4326").to_file('coastchange_hotspots/hotspots_pop_1000_buffered.geojson')

In [119]:
hotspots_pop1000_buffered.sort_values(by='total_pop')

,pop,geometry,total_pop,GID_0,NAME_0,GID_1,NAME_1
0,1.0,"POLYGON ((1059057.534 4212635.431, 1059084.335...",0.871831,TUN,Tunisia,TUN.17_1,Sfax
1,1.0,"POLYGON ((1059003.930 4212606.155, 1059030.732...",0.871834,TUN,Tunisia,TUN.17_1,Sfax
4,1.0,"POLYGON ((1059084.335 4212576.879, 1059111.137...",0.871837,TUN,Tunisia,TUN.17_1,Sfax
2,1.0,"POLYGON ((1058441.093 4212576.879, 1058467.895...",0.871837,TUN,Tunisia,TUN.17_1,Sfax
3,1.0,"POLYGON ((1059030.732 4212576.879, 1059057.534...",0.871837,TUN,Tunisia,TUN.17_1,Sfax
...,...,...,...,...,...,...,...
55,10.0,"POLYGON ((415926.607 813890.060, 419225.231 81...",4921.669803,NGA,Nigeria,NGA.29_1,Ondo
100,10.0,"POLYGON ((420615.777 808496.909, 423914.397 81...",4922.039716,NGA,Nigeria,NGA.29_1,Ondo
69,10.0,"POLYGON ((420829.662 747138.869, 420575.311 74...",4984.747101,NGA,Nigeria,NGA.29_1,Ondo
2,94.0,"POLYGON ((852428.139 44597.226, 852668.902 494...",7737.354513,GAB,Gabon,GAB.1_1,Estuaire


In [120]:
total=hotspots_pop1000.groupby('NAME_1').sum()[['total_pop']]
total.sort_values(by='total_pop', ascending=False)

,total_pop
NAME_1,
Ondo,41474.089289
Littoral,29129.659365
Lagos,16714.585484
Estuaire,15300.592224
Rivers,13619.932059
...,...
Misratah,5.383388
Al Bahr al Ahmar,3.745006
Western,3.083326


In [121]:
count = hotspots_pop1000_buffered.groupby('NAME_0').count()[['NAME_1']]
print(len(count))
count.sort_values(by='NAME_1', ascending=False).rename(columns ={'NAME_1':'Count'})

26


,Count
NAME_0,
Mozambique,570
Nigeria,551
Togo,278
Sierra Leone,265
Guinea,234
Madagascar,207
Senegal,159
Benin,138
Ghana,118


In [122]:
hotspots_pop1000["NAME_1"] = hotspots_pop1000["NAME_0"]+":"+hotspots_pop1000["NAME_1"]
count = hotspots_pop1000.groupby('NAME_1').count()[['NAME_0']]
print(len(count))
count.sort_values(by='NAME_0', ascending=False).rename(columns ={'NAME_0':'Count'})

80


,Count
NAME_1,
Mozambique:Sofala,419
Togo:Maritime,278
Nigeria:Lagos,234
Sierra Leone:Northern,205
Guinea:Boké,163
...,...
Tanzania:Mjini Magharibi,1
Egypt:Matrouh,1
Gambia:Western,1


In [124]:
gadm_level1=gadm_level1.join(hotspots_pop1000[["total_pop", "GID_1"]], level)



NameError: name 'level' is not defined

In [ ]:
param = 'perc_neg1'
title = 'Percentage of shorelines retreated at more than 1 meter per year'

print(gid_1_coastalchange[param].min(), gid_1_coastalchange[param].max())

steps = 5 #np.ceil((gid_1_coastalchange[param].max()/5)/10)*10
vmin, vmax = 0, 55 #np.ceil(gid_1_coastalchange[param].max()/steps)*steps+steps
print(vmin, vmax)

# Define plot and colourbar axes
fig, ax = plt.subplots(1,1, figsize=(15,15))
fig.subplots_adjust(bottom=0.2)
cax = fig.add_axes([0.16, 0.15, 0.70, 0.03])


# Define colour map
cmap = mpl.cm.YlOrRd
bounds = list(np.arange(vmin,vmax,steps))
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)#, extend='both')
cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap,
                                norm=norm,
                                orientation='horizontal')

# Define colourbar labelling
# cbar.set_ticks([])
cbar.set_ticks(bounds)
ticklabels= list('{:.0f}'.format(i) for i in bounds)
ticklabels[-1]=f">{ticklabels[-1]}"
cbar.set_ticklabels(ticklabels)
cbar.set_label(title, fontsize='14')

# Turn off lon-lat ticks and labels
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_yticks([])

# # Remove frame
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

gid_1_coastalchange.plot(param, ax=ax, cmap=cmap, norm=norm);
#gid_1_coastalchange.geometry.plot(ax=ax, linewidth=0.2, edgecolor='black', facecolor="none");
#gid_0_coastalchange.simplify(0.05).geometry.plot(ax=ax, linewidth=0.2, edgecolor='black', facecolor="none");
african_countries.geometry.plot(ax=ax, linewidth=0.2, edgecolor='black', facecolor="none");


fig.savefig(f"GID_{level}_{param}.png", bbox_inches='tight')